## Урок 1. Концепции хранения информации

### Практическая работа

1. Каждое из слов «разработка», «сокет», «декоратор» представить в строковом формате и проверить тип и содержание соответствующих переменных. Затем с помощью онлайн-конвертера преобразовать строковое представление в формат Unicode и также проверить тип и содержимое переменных.

In [46]:
data1 = "разработка"
print(data1)
print(type(data1))

data1u = "\u0440\u0430\u0437\u0440\u0430\u0431\u043e\u0442\u043a\u0430"
print(data1u)
print(type(data1u))

разработка
<class 'str'>
разработка
<class 'str'>


In [48]:
data2 = "сокет"
print(data2)
print(type(data2))

data2u = "\u0441\u043e\u043a\u0435\u0442"
print(data2u)
print(type(data2u))

сокет
<class 'str'>
сокет
<class 'str'>


In [49]:
data3 = "декоратор"
print(data3)
print(type(data3))

data3u = "\u0434\u0435\u043a\u043e\u0440\u0430\u0442\u043e\u0440"
print(data3u)
print(type(data3u))

декоратор
<class 'str'>
декоратор
<class 'str'>


2. Каждое из слов «class», «function», «method» записать в байтовом типе без преобразования в последовательность кодов (не используя методы encode и decode) и определить тип, содержимое и длину соответствующих переменных.

In [43]:
data4 = b"class"
print(data4)
print(type(data4))
print(len(data4))

b'class'
<class 'bytes'>
5


In [44]:
data5 = b"function"
print(data5)
print(type(data5))
print(len(data5))

b'function'
<class 'bytes'>
8


In [45]:
data6 = b"method"
print(data6)
print(type(data6))
print(len(data6))

b'method'
<class 'bytes'>
6


3. Определить, какие из слов «attribute», «класс», «функция», «type» невозможно записать в байтовом типе.

In [52]:
data7 = b"attribute"

In [53]:
data8 = b"класс" # невозможно записать в байтовом типе напрямую, так как нет прямого соответствия в байтах для кириллицы 

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-53-1c40f1706954>, line 1)

In [54]:
data9 = b"функция" # невозможно записать в байтовом типе напрямую, так как нет прямого соответствия в байтах для кириллицы 

SyntaxError: bytes can only contain ASCII literal characters. (<ipython-input-54-8dd81d092341>, line 1)

In [55]:
data10 = b"type"

4. Преобразовать слова «разработка», «администрирование», «protocol», «standard» из строкового представления в байтовое и выполнить обратное преобразование (используя методы encode и decode).

In [56]:
data11 = "разработка"
data11b = data11.encode()
print(data11b)
print(type(data11b))
data11s = data11b.decode()
print(data11s)
print(type(data11s))

b'\xd1\x80\xd0\xb0\xd0\xb7\xd1\x80\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x82\xd0\xba\xd0\xb0'
<class 'bytes'>
разработка
<class 'str'>


In [57]:
data12 = "администрирование"
data12b = data12.encode()
print(data12b)
print(type(data12b))
data12s = data12b.decode()
print(data12s)
print(type(data12s))

b'\xd0\xb0\xd0\xb4\xd0\xbc\xd0\xb8\xd0\xbd\xd0\xb8\xd1\x81\xd1\x82\xd1\x80\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5'
<class 'bytes'>
администрирование
<class 'str'>


In [58]:
data13 = "protocol"
data13b = data13.encode()
print(data13b)
print(type(data13b))
data13s = data13b.decode()
print(data13s)
print(type(data13s))

b'protocol'
<class 'bytes'>
protocol
<class 'str'>


In [59]:
data14 = "standard"
data14b = data14.encode()
print(data14b)
print(type(data14b))
data14s = data14b.decode()
print(data14s)
print(type(data14s))

b'standard'
<class 'bytes'>
standard
<class 'str'>


5. Выполнить пинг веб-ресурсов yandex.ru, youtube.com и преобразовать результаты из байтовового в строковый тип на кириллице.

6. Создать текстовый файл test_file.txt, заполнить его тремя строками: «сетевое программирование», «сокет», «декоратор». Проверить кодировку файла по умолчанию. Принудительно открыть файл в формате Unicode и вывести его содержимое.

In [60]:
import locale
def_coding = locale.getpreferredencoding()
print(def_coding)

cp1251


In [61]:
f_n = open("test_file.txt", "w")
f_n.write("сетевое программирование\nсокет\nдекоратор")
f_n.close()
print(f_n)

<_io.TextIOWrapper name='test_file.txt' mode='w' encoding='cp1251'>


In [64]:
with open("test_file.txt", encoding="cp1251") as f_n:
    for el_str in f_n:
        print(el_str, end='')

сетевое программирование
сокет
декоратор

In [65]:
with open("test_file.txt", encoding="utf-8") as f_n:
    for el_str in f_n:
        print(el_str, end='')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 0: invalid continuation byte

### Необходимые и достаточные условия

1. Реализовать приведение строк "разработка", "сокет", "декоратор" к типу bytes используя нативные методы строк;

In [14]:
data1 = "разработка"
data1b = data1.encode()
print(data1b)
print(type(data1b))

b'\xd1\x80\xd0\xb0\xd0\xb7\xd1\x80\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x82\xd0\xba\xd0\xb0'
<class 'bytes'>


In [15]:
data2 = "сокет"
data2b = data2.encode()
print(data2b)
print(type(data2b))

b'\xd1\x81\xd0\xbe\xd0\xba\xd0\xb5\xd1\x82'
<class 'bytes'>


In [16]:
data3 = "декоратор"
data3b = data3.encode()
print(data3b)
print(type(data3b))

b'\xd0\xb4\xd0\xb5\xd0\xba\xd0\xbe\xd1\x80\xd0\xb0\xd1\x82\xd0\xbe\xd1\x80'
<class 'bytes'>


2. Реализовать приведение полученных экземпляров типа bytes к типу str;

In [29]:
data1s = data1b.decode()
print(data1s)
print(type(data1s))

разработка
<class 'str'>


In [30]:
data2s = data2b.decode()
print(data2s)
print(type(data2s))

сокет
<class 'str'>


In [31]:
data3s = data3b.decode()
print(data3s)
print(type(data3s))

декоратор
<class 'str'>


3. Реализовать приведение полученных строк и байтовых последовательностей с использование различных кодировок utf-8 latin-1.

In [26]:
data1s_enc = data1s.encode('utf-8')
print(data1s_enc.decode('latin-1'))

ÑÐ¾ÐºÐµÑ


In [27]:
data2s_enc = data2s.encode('utf-8')
print(data2s_enc.decode('latin-1'))

ÑÐ¾ÐºÐµÑ


In [28]:
data3s_enc = data3s.encode('utf-8')
print(data3s_enc.decode('latin-1'))

Ð´ÐµÐºÐ¾ÑÐ°ÑÐ¾Ñ


In [33]:
data1b_dec = data1b.decode('latin-1')
print(data1b_dec)

ÑÐ°Ð·ÑÐ°Ð±Ð¾ÑÐºÐ°


In [34]:
data2b_dec = data2b.decode('latin-1')
print(data2b_dec)

ÑÐ¾ÐºÐµÑ


In [35]:
data3b_dec = data3b.decode('latin-1')
print(data3b_dec)

Ð´ÐµÐºÐ¾ÑÐ°ÑÐ¾Ñ
